<a href="https://colab.research.google.com/github/deepak1195/DeepLearning/blob/main/009_StochasticVsBatchVsMiniGradientDescent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras

import plotly.express as px
import plotly.graph_objects as go

import random

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path="/content/drive/MyDrive/Study/Data/"

Mounted at /content/drive


In [ ]:
df=pd.read_csv(f'{path}housePrices.csv')
df

,area,bedrooms,price
0,1056,2,39.07
1,2600,4,120.00
2,1440,3,62.00
3,1521,3,75.00
4,1200,2,51.00
5,1170,2,38.00
6,2732,4,135.00
7,3300,4,155.00
8,1310,3,50.00
9,3700,5,167.00


In [ ]:
def scaling(sDf):
  for f in sDf:
    sDf[f]=(sDf[f]-sDf[f].min())/(sDf[f].max()-sDf[f].min())
  return sDf

In [ ]:
def trainTestSplit(mDf,yCol,sF=0.2):
  import random
  mDf=scaling(mDf.copy())
  idx=random.sample(range(0, mDf.shape[0]),int(mDf.shape[0]*(1-sF)))
  train=mDf[df.index.isin(idx)]
  test=mDf[~df.index.isin(idx)]
  xCol=mDf.columns[~mDf.columns.isin([yCol])]
  return train[xCol],train[yCol],test[xCol],test[yCol]

xTrain,yTrain,xTest,yTest=trainTestSplit(df,'price',sF=0.2)

In [ ]:
xTrain

,area,bedrooms
0,0.088276,0.25
1,0.620690,0.75
3,0.248621,0.50
4,0.137931,0.25
7,0.862069,0.75
8,0.175862,0.50
9,1.000000,1.00
10,0.344828,0.50
11,0.684483,0.75
12,0.068966,0.25


### Batch Gradient Descent

In [ ]:
class BGDNN():
  def __init__(self):
    self.weigPlusBias=None
    self.cost=list()
    self.epoch=list()
  def logLoss(self,yT,yP):
    eplison=1e-15
    nYp=[max(i,eplison) for i in yP]
    nYp=np.array([min(i,1-eplison) for i in nYp])
    return -np.mean(yT*np.log(nYp)+(1-yT)*np.log(1-nYp))

  def MSE(self,true,pre):
    return np.mean(np.square(true-pre))

  def sigmoid(self,x):
    return 1/(1+np.exp(-x))

  def gradDesc(self, xT,yT,epochs,lThr):
    ws=np.ones(xT.shape[1])
    xs=np.array(xT.columns)
    b=np.zeros(xT.shape[0])
    r=0.5
    n=len(yT)
    for e in range(epochs):
      eq=[b]
      for i,j in zip(range(0,len(xs)),xs):
        eq.append(np.array(ws[i]*xT[[j]].values).reshape([1, n])[0])

      weigSum=np.sum(np.array(eq), axis = 0)
      yP=self.sigmoid(weigSum)
      loss=self.MSE(yT,yP)

      self.cost.append(loss)
      self.epoch.append(e)

      for w,x in zip(range(len(ws)),xs):
        ws[w]=ws[w]-r*((1/n)*np.dot(np.transpose(xT[x]),(yP-yT)))
      b=b-r*(np.mean(yP-yT))

      if e%50==0:
        print(f"Epoch : {e}/{epochs}| Weights : {ws}| Bias : {b[0]}| Loss : {loss}")

      if loss<=lThr:
        print(f"Epoch : {e}/{epochs}| Weights : {ws}| Bias : {b[0]}| Loss : {loss}")
        break

    return np.append(ws,b[0])

  def fit(self,x,y,ep,lTh=0.0):
    self.weigPlusBias=self.gradDesc(x,y,ep,lTh)

  def predict(self,xTest):
    n=len(xTest)
    pws=[]
    for i,j in zip(range(0,len(xTest)),xTest):
      pws.append(np.array(self.weigPlusBias[i]*xTest[j].values))

    pws.append(np.ones(xTest.shape[0])*self.weigPlusBias[-1])
    return self.sigmoid(np.sum(np.array(pws), axis = 0))

In [ ]:
n=BGDNN()

In [ ]:
n.fit(xTrain,yTrain,501)

Epoch : 0/501| Weights : [0.97226388 0.94393762]| Bias : -0.16217610075343206| Loss : 0.15458383915260182
Epoch : 50/501| Weights : [1.75057734 1.14856925]| Bias : -1.6974782515026878| Loss : 0.023304211016186842
Epoch : 100/501| Weights : [2.42661362 1.5086665 ]| Bias : -2.193713491121271| Loss : 0.01130796494129961
Epoch : 150/501| Weights : [2.88481694 1.75384346]| Bias : -2.5309844396456453| Loss : 0.006717409698502473
Epoch : 200/501| Weights : [3.21217713 1.93299927]| Bias : -2.7741503745297442| Loss : 0.004704135110359013
Epoch : 250/501| Weights : [3.45390953 2.07027947]| Bias : -2.9564313518881957| Loss : 0.0037514606597284553
Epoch : 300/501| Weights : [3.63626095 2.17913349]| Bias : -3.0968286334727937| Loss : 0.0032850707164454444
Epoch : 350/501| Weights : [3.77573689 2.26773919]| Bias : -3.2071372752039484| Loss : 0.0030574331196024057
Epoch : 400/501| Weights : [3.88334378 2.34140044]| Bias : -3.2951389401890747| Loss : 0.002951836363675904
Epoch : 450/501| Weights : [3.

In [ ]:
px.line(x=n.epoch,y=n.cost)

In [ ]:
pre=n.predict(xTest)
pre

array([0.21318234, 0.0914952 , 0.75111496, 0.08372143])

In [ ]:
yTest

2     0.222222
5     0.044444
6     0.762963
13    0.059259
Name: price, dtype: float64

In [ ]:
df.iloc[yTest.index]['price']

2      62.0
5      38.0
6     135.0
13     40.0
Name: price, dtype: float64

In [ ]:
from sklearn import preprocessing
sy=preprocessing.MinMaxScaler()
scaledY=sy.fit_transform(df['price'].values.reshape(df.shape[0],1))
sy.inverse_transform([pre])

array([[ 60.77961613,  44.35185173, 133.40051955,  43.30239349]])

### Stochastic Gradient Descent

In [ ]:
class SGDNN():
  def __init__(self):
    self.weigPlusBias=None
    self.cost=list()
    self.epoch=list()
  def logLoss(self,yT,yP):
    eplison=1e-15
    nYp=[max(i,eplison) for i in yP]
    nYp=np.array([min(i,1-eplison) for i in nYp])
    return -np.mean(yT*np.log(nYp)+(1-yT)*np.log(1-nYp))

  def MSE(self,true,pre):
    return np.mean(np.square(true-pre))

  def sigmoid(self,x):
    return 1/(1+np.exp(-x))

  def gradDesc(self, xT,yT,epochs,lThr):
    ws=np.ones(xT.shape[1])
    xs=np.array(xT.columns)
    b=0
    r=0.5
    n=len(yT)
    for e in range(epochs):
      eq=[b]
      si=random.randint(0,n-1)
      for i,j in zip(range(0,len(xs)),xs):
        eq.append(ws[i]*xT[[j]][si:si+1].values[0][0])
      weigSum=np.sum(np.array(eq), axis = 0)
      yP=self.sigmoid(weigSum)
      loss=self.MSE(yT,yP)

      self.cost.append(loss)
      self.epoch.append(e)

      for w,x in zip(range(len(ws)),xs):
        ws[w]=ws[w]-r*((1/n)*np.dot(np.transpose(xT[x][si:si+1].values),(yP-yT[si:si+1].values)))
      b=b-r*(np.mean(yP-yT[si:si+1].values))

      if e%10==0:
        print(f"Epoch : {e}/{epochs}| Weights : {ws}| Bias : {b}| Loss : {loss}")

      if loss<=lThr:
        print(f"Epoch : {e}/{epochs}| Weights : {ws}| Bias : {b}| Loss : {loss}")
        break

    return np.append(ws,b)

  def fit(self,x,y,ep,lTh=0.0):
    self.weigPlusBias=self.gradDesc(x,y,ep,lTh)

  def predict(self,xTest):
    n=len(xTest)
    pws=[]
    for i in range(0,n):
      pws.append(np.array(self.weigPlusBias[i]*xTest[i]))
    pws.append(self.weigPlusBias[-1])
    print(pws)
    return self.sigmoid(np.sum(np.array(pws), axis = 0))

In [ ]:
sn=SGDNN()
sn.fit(xTrain,yTrain,100)

Epoch : 0/100| Weights : [0.99622905 0.99261098]| Bias : -0.23644864438504554| Loss : 0.20067845579541715
Epoch : 10/100| Weights : [0.98508483 0.96869405]| Bias : -1.4665083433118247| Loss : 0.10304017874589638
Epoch : 20/100| Weights : [1.00914078 0.98298529]| Bias : -1.5477771928885244| Loss : 0.1166446724414934
Epoch : 30/100| Weights : [1.02486621 0.99381065]| Bias : -1.605315178916532| Loss : 0.11697881755249197
Epoch : 40/100| Weights : [1.04189307 1.00417807]| Bias : -1.558197873135376| Loss : 0.10283202284955809
Epoch : 50/100| Weights : [1.06901787 1.02747039]| Bias : -1.367916694950034| Loss : 0.16990215567744094
Epoch : 60/100| Weights : [1.09384857 1.04661349]| Bias : -1.2060258547495875| Loss : 0.1061228874044525
Epoch : 70/100| Weights : [1.09047855 1.03658197]| Bias : -1.6466258716547273| Loss : 0.10442025844556185
Epoch : 80/100| Weights : [1.10517726 1.04422775]| Bias : -1.617558725891428| Loss : 0.10354772236086478
Epoch : 90/100| Weights : [1.12205131 1.05684496]| B

In [ ]:
sn.weigPlusBias

array([ 1.15025723,  1.08202373, -1.28531838])

In [ ]:
px.line(x=sn.epoch,y=sn.cost)

In [ ]:
xTest

,area,bedrooms
2,0.220690,0.50
5,0.127586,0.25
6,0.666207,0.75
13,0.103448,0.25


In [ ]:
sy.inverse_transform([[sn.predict([	0.344828,	0.50])]])

[array(0.3966409), array(0.54101186), -1.2853183776317199]


array([[87.88306346]])

### Mini Batch Gradient Descent

In [ ]:
class MBGDNN():
  def __init__(self):
    self.weigPlusBias=None
    self.cost=list()
    self.epoch=list()
  def logLoss(self,yT,yP):
    eplison=1e-15
    nYp=[max(i,eplison) for i in yP]
    nYp=np.array([min(i,1-eplison) for i in nYp])
    return -np.mean(yT*np.log(nYp)+(1-yT)*np.log(1-nYp))

  def MSE(self,true,pre):
    return np.mean(np.square(true-pre))

  def sigmoid(self,x):
    return 1/(1+np.exp(-x))

  def gradDesc(self, xT,yT,epochs,lThr):
    ws=np.ones(xT.shape[1])
    xs=np.array(xT.columns)
    b=np.zeros(xT.shape[0])
    r=0.5
    n=len(yT)
    for e in range(epochs):
      idx=xT.index.values[random.sample(range(0,n),3)]
      eq=[b[[vi for vi in range(len(idx))]]]
      for i,j in zip(range(0,len(xs)),xs):
        eq.append(np.array(ws[i]*xT[xT.index.isin(idx)][[j]].values).reshape([1, len(idx)])[0])
      weigSum=np.sum(np.array(eq), axis = 0)
      yP=self.sigmoid(weigSum)
      loss=self.MSE(yT[yT.index.isin(idx)],yP)

      self.cost.append(loss)
      self.epoch.append(e)

      for w,x in zip(range(len(ws)),xs):
        ws[w]=ws[w]-r*((1/n)*np.dot(np.transpose(xT[xT.index.isin(idx)][x]),(yP-yT[xT.index.isin(idx)])))
      b=b-r*(np.mean(yP-yT[xT.index.isin(idx)]))

      if e%50==0:
        print(f"Epoch : {e}/{epochs}| Weights : {ws}| Bias : {b[0]}| Loss : {loss}")

      if loss<=lThr:
        print(f"Epoch : {e}/{epochs}| Weights : {ws}| Bias : {b[0]}| Loss : {loss}")
        break

    return np.append(ws,b[0])

  def fit(self,x,y,ep,lTh=0.0):
    self.weigPlusBias=self.gradDesc(x,y,ep,lTh)

  def predict(self,xTest):
    n=len(xTest)
    pws=[]
    for i,j in zip(range(0,len(xTest)),xTest):
      pws.append(np.array(self.weigPlusBias[i]*xTest[j].values))

    pws.append(np.ones(xTest.shape[0])*self.weigPlusBias[-1])
    return self.sigmoid(np.sum(np.array(pws), axis = 0))

In [ ]:
mn=MBGDNN()
mn.fit(xTrain,yTrain,200)

Epoch : 0/200| Weights : [0.99344971 0.98883151]| Bias : -0.20160315689163685| Loss : 0.18042283276787865
Epoch : 50/200| Weights : [1.20260892 1.08016482]| Bias : -1.5867898703194265| Loss : 0.05060150903062394
Epoch : 100/200| Weights : [1.4205437  1.22191975]| Bias : -1.672112750324699| Loss : 0.015260583945820354
Epoch : 150/200| Weights : [1.64036385 1.37157952]| Bias : -1.7800486405090277| Loss : 0.031006192692982503


In [ ]:
mn.weigPlusBias

array([ 1.84592765,  1.51732805, -1.81262936])

In [ ]:
px.line(x=mn.epoch,y=mn.cost)